In [12]:
# Import necessary libraries
from sklearn.linear_model import *
from sklearn.metrics import *
from sklearn.linear_model import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.model_selection import *
from sklearn.preprocessing import *
from sklearn.neighbors import KNeighborsClassifier,KNeighborsRegressor
from sklearn.pipeline import *
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import warnings
from sklearn.linear_model import *
import os

In [13]:
# Ignore warnings
warnings.simplefilter("ignore")

# Change the current working directory to "D:\kaggla_comp\hardness"
os.chdir(r"D:\kaggla_comp\hardness")

# Load the training data into a DataFrame named "mohs"
mohs = pd.read_csv("train.csv", index_col=0)

# Extract the feature variables "X" (all columns except 'Hardness') and the target variable "y" (Hardness)
x = mohs.drop('Hardness', axis=1)
y = mohs['Hardness']
mohs

,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average,Hardness
id,,,,,,,,,,,,
0,100.0,0.841611,10.000000,4.800000,20.612526,11.088100,2.766000,1.732000,0.860000,0.496070,0.91457,6.0
1,100.0,7.558488,10.000000,4.800000,20.298893,12.040830,2.755000,1.631000,0.910000,0.492719,0.71760,6.5
2,76.0,8.885992,15.600000,5.600000,33.739258,12.086300,2.828000,1.788000,0.864000,0.481478,1.50633,2.5
3,100.0,8.795296,10.000000,4.800000,20.213349,10.948500,2.648000,1.626000,0.936000,0.489272,0.78937,6.0
4,116.0,9.577996,11.600000,4.800000,24.988133,11.824480,2.766000,1.682000,0.896000,0.492736,1.86481,6.0
...,...,...,...,...,...,...,...,...,...,...,...,...
10402,128.0,7.558488,12.000000,4.000000,26.385218,11.330440,2.644000,1.631000,0.892000,0.496070,1.79607,4.0
10403,30.0,1.743160,10.000000,5.333333,20.766935,14.163933,3.090000,1.556667,0.866667,0.480390,0.81480,5.0
10404,196.0,30.920000,24.500000,5.500000,53.490297,10.074300,2.295000,1.545000,1.120000,0.469715,2.11540,1.8


In [14]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3, random_state=24)

knn = KNeighborsRegressor()  # regressor
knn.fit(x_train,y_train)
y_pred = knn.predict(x_test)
print('r2 score(without scaling): ',r2_score(y_test,y_pred))

r2 score(without scaling):  0.29858393602731925


# Standard scaling

In [15]:
std_scaler = StandardScaler()

pipe = Pipeline([('SCL',std_scaler),('KNN',knn)])
pipe.fit(x_train,y_train)
y_pred = pipe.predict(x_test)
print('r2 score(with scaling): ',r2_score(y_test,y_pred))

r2 score(with scaling):  0.3484152621679638


# ALl in one

In [16]:
mn_scaler = MinMaxScaler()
std_scaler = StandardScaler()
knn = KNeighborsRegressor()
pipe = Pipeline([('SCL',mn_scaler),('KNN',knn)])
params= {'KNN__n_neighbors':np.arange(1,25),
         'SCL':[None,std_scaler,mn_scaler]} #(dunder)
kfold = KFold(n_splits=5,shuffle=True,random_state=24)
gcv = GridSearchCV(pipe,param_grid=params,cv =kfold)
gcv.fit(x,y)  #(gcv.fit actually a loop)

print(gcv.best_params_)
print(gcv.best_score_)

{'KNN__n_neighbors': 23, 'SCL': StandardScaler()}
0.4023554773402032


# Inferencing

In [17]:
best_model = gcv.best_estimator_

# Load the test data into a DataFrame named "test"
test = pd.read_csv("test.csv", index_col=0)

pred_hard = best_model.predict(test)

# Load the sample submission file into a DataFrame named "submit"
submit = pd.read_csv("sample_submission.csv")

# Replace the predicted hardness values in the submission file
submit['Hardness'] = pred_hard

# Save the submission file with the predicted hardness values
submit.to_csv("D:\kaggla_comp\hardness\sbt_rg_19_jun.csv", index=False)